In [1]:
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence
import random
from bilstm_crf import BiLSTM_CRF
from load_data_crf import num_words, num_tags, words, tags, word2idx, tag2idx, train_X, train_y, test_X, test_y

In [2]:
new_model = BiLSTM_CRF(num_words, tag2idx, embedding_dim=50, hidden_dim=200)
new_model.load_state_dict(torch.load("./weights/bilstm_crf.pth", map_location='cpu'))

C:\Users\ziyet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\nn\modules\rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


<All keys matched successfully>

In [3]:

new_model.eval()
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(len(test_X)):
    cur_X = torch.tensor(test_X[i:i+1], dtype = torch.long)
    cur_y = torch.tensor(test_y[i], dtype = torch.long)

    _, pred = new_model(cur_X)
    pred = torch.tensor(pred[0])
    TP += torch.sum((pred!=tag2idx['O']) * (pred == cur_y))
    TN += torch.sum((pred==tag2idx['O']) * (pred == cur_y))
    FP += torch.sum((pred!=tag2idx['O']) * (pred != cur_y))
    FN += torch.sum((pred==tag2idx['O']) * (pred != cur_y))
    
    if (i+1)%1000 == 0:
        print(i)

prec = TP/(TP+FP)
recall = TP/(TP+FN)
f1 = prec*recall/(prec+recall)
print("Precision: ",prec)
print("Recall: ", recall)
print("F1 Score: ", f1*2)

999
1999
2999
3999
4999
5999
6999
7999
8999
Precision:  tensor(0.8039)
Recall:  tensor(0.9244)
F1 Score:  tensor(0.8599)


In [4]:
new_model.eval()
for i in range(7990, 8000):
    cur_X = torch.tensor(test_X[i:i+1], dtype = torch.long)
    cur_y = torch.tensor(test_y[i:i+1], dtype = torch.long)

    _, pred = new_model(cur_X)
    pred_tag = ""
    t_tag = ""
    sentence = ""
    for j in range(0, len(cur_X[0])):
        sentence += words[cur_X[0][j]]+" "
        t_tag += tags[cur_y[0][j]] + " "
        pred_tag += tags[pred[0][j]] + " "
    
    print(sentence,'\n')
    print("labels: ", t_tag,'\n')
    print("predictions:", pred_tag,'\n\n')

The famine killed as many as 10 million people in what was then Soviet Ukraine .  

labels:  O O O O O O O O O O O O O O B-geo O  

predictions: O O O O O O O O O O O O O O B-geo O  


Many analysts say the famine was not from natural causes , but instead was the result of government policies under Soviet dictator Josef Stalin .  

labels:  O O O O O O O O O O O O O O O O O O O O O O B-per I-per O  

predictions: O O O O O O O O O O O O O O O O O O O O O O B-per I-per O  


Ukrainian President Viktor Yushchenko supported the bill .  

labels:  B-gpe B-per I-per I-per O O O O  

predictions: B-gpe B-per I-per I-per O O O O  


Most have been released unharmed , often after a payment of ransom .  

labels:  O O O O O O O O O O O O O  

predictions: O O O O O O O O O O O O O  


On Saturday , he spoke in the capital , Kiev , to mark the 73rd anniversary of the tragedy , known in Ukraine as the Holodomor .  

labels:  O B-tim O O O O O O O B-per O O O O O O O O O O O O B-geo O O B-geo O  


In [5]:
import torch
vec = torch.randn((3,4))
print(vec)
i=1
max_score = vec[i, torch.argmax(vec[i])]
print(max_score)
print(torch.max(vec[i]))
max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
print(max_score_broadcast)
print(max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast))))
print(max_score + torch.log(torch.sum(torch.exp(vec - max_score))))

tensor([[-0.0460,  0.3587,  1.2541,  0.1426],
        [ 0.1849, -0.1925,  0.7593,  0.4366],
        [ 1.8217,  0.6672, -1.4221, -2.5612]])
tensor(0.7593)
tensor(0.7593)
tensor([[0.7593, 0.7593, 0.7593, 0.7593]])
tensor(3.0543)
tensor(3.0543)
